<a href="https://colab.research.google.com/github/paulsiddhartha0/travelling_salesman_problem/blob/main/TSP_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 33.3 MB/s 
     |████████████████████████████████| 408 kB 76.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.9 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.9 which is incompatible.
google-api-core 1.31

In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 36.4 MB/s 


In [ ]:
import pandas as pd
from math import cos, asin, sqrt
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import xlsxwriter
from collections import defaultdict

In [ ]:
def DistanceMatrix(x1, y1, x2, y2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((x2 - x1) * p)/2 + cos(x1 * p) * cos(x2 * p) * (1 - cos((y2 - y1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [ ]:
import pandas as pd
df = pd.read_csv('India Cities LatLng.csv')
df = df[df.city!= 'Kolkata']
cities_lat_dict = df[['city', 'lat']].set_index('city')['lat'].to_dict()
cities_lng_dict = df[['city', 'lng']].set_index('city')['lng'].to_dict()

In [ ]:
def create_data_model():
  nodes = [0] ## initializing with the depot node as 0
  lat = {}
  lng = {}
  original_id_dict = {}

  depot_node = [0]
  original_id_dict[0] = 'Kolkata'
  lat[0] = 22.5411
  lng[0] = 88.3378


  cust_loc = 1
  for city in cities_lat_dict.keys():
    lat[cust_loc] = cities_lat_dict[city]
    lng[cust_loc] = cities_lng_dict[city]
    original_id_dict[cust_loc] = city
    nodes.append(cust_loc)
    cust_loc +=1

  location = {} 
  distance = {}
  for from_node in nodes:
    location[from_node] = (lat[from_node], lng[from_node])
    distance[from_node] = {}
    for to_node in nodes:
        distance[from_node][to_node] = DistanceMatrix(lat[from_node], lng[from_node],
                                          lat[to_node], lng[to_node])
        
  data = {}
  data["locations"] = location
  data["num_locations"] = len(data["locations"])
  data["num_vehicles"] = 1
  data["depot"] = 0
  data["distance_matrix"] = distance
  data["original_id_dict"] = original_id_dict
  return data





In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_nodes_visited = 0
    route_batch_info = defaultdict(list)
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            plan_output += ' {} -> '.format(data["original_id_dict"][node_index])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            if node_index != 0:
                total_nodes_visited += 1
            route_batch_info[vehicle_id].append(data["original_id_dict"][node_index])
        node_index = manager.IndexToNode(index)
        plan_output += '{}\n'.format(data["original_id_dict"][node_index])
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        route_batch_info[vehicle_id].append(data["original_id_dict"][node_index])

        if route_distance > 0:
            print(plan_output)
            total_distance += route_distance
    print('Total Distance of all routes: {}KM'.format(total_distance))
    print('\n Cities not visited = ', len(data['locations']) - total_nodes_visited-1 )

    workbook = xlsxwriter.Workbook('{}.xlsx'.format('route_plot'))

    worksheet = workbook.add_worksheet('Route_Info_Plot')
    merge_format = workbook.add_format({
        'bold': True,
        'align': 'center',
    })
    format_c = workbook.add_format({'align': 'center'})
    worksheet.write(0, 0, 'Rider_ID', merge_format)
    worksheet.write(0, 1, 'Route', merge_format)

    row = 1
    worksheet.write(row, 0, 0, format_c)
    worksheet.write(row, 1, str(route_batch_info[0]), format_c)



    worksheet = workbook.add_worksheet('Cities_lat_long')
    merge_format = workbook.add_format({
        'bold': True,
        'align': 'center',
    })
    format_c = workbook.add_format({'align': 'center'})
    worksheet.write(0, 0, 'id', merge_format)
    worksheet.write(0, 1, 'lat', merge_format)
    worksheet.write(0, 2, 'long', merge_format)
    row = 1
    for i in data['locations'].keys():
        # if len(route_info_plot[b]) > 2:
        worksheet.write(row, 0, data['original_id_dict'][i], format_c)
        worksheet.write(row, 1, data['locations'][i][0], format_c)
        worksheet.write(row, 2, data['locations'][i][1], format_c)
        row = row + 1

    workbook.close()

In [ ]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Improve the initial solution by a meta-heuristic algorithm

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.time_limit.FromSeconds(10)
    search_parameters.time_limit.seconds = 50
    search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [ ]:
if __name__ == '__main__':
    main()

Objective: 21494
Route for vehicle 0:
 Kolkata ->  Uluberiya ->  Bārāsat ->  Agartala ->  Āīzawl ->  Imphāl ->  Kohīma ->  Itānagar ->  Shillong ->  Dispur ->  Guwahati ->  Gangtok ->  Shiliguri ->  Purnea ->  Bhāgalpur ->  Begusarai ->  Muzaffarpur ->  Patna ->  Gaya ->  Ranchi ->  Raurkela ->  Sambalpur ->  Bilāspur ->  Raipur ->  Bhilai ->  Drug ->  Jabalpur ->  Sannai ->  Allahabad ->  Mirzapur ->  Varanasi ->  Gorakhpur ->  Lucknow ->  Cawnpore ->  Etāwah ->  Fīrozābād ->  Agra ->  Bharatpur ->  Mathura ->  Aligarh ->  Shāhjānpur ->  Bareilly ->  Rāmpur ->  Moradabad ->  Sambhal ->  Hāpur ->  Meerut ->  Muzaffarnagar ->  Pānīpat ->  Karnāl ->  Sahāranpur ->  Dehra Dūn ->  Panchkula ->  Chandigarh ->  Shimla ->  Srinagar ->  Handwāra ->  Jammu ->  Amritsar ->  Jalandhar ->  Ludhiāna ->  Patiāla ->  Hisar ->  Rohtak ->  Sonīpat ->  New Delhi ->  Delhi ->  Ghaziabad ->  Faridabad ->  Alwar ->  Sīkar ->  Bīkaner ->  Jodhpur ->  Pāli ->  Udaipur ->  Bhīlwāra ->  Kota ->  Ajmer ->  Jaip

# Plotting in Kepler GL

In [ ]:
!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 37.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 59.8 MB/s 
     |████████████████████████████████| 16.7 MB 70.7 MB/s 
     |████████████████████████████████| 6.3 MB 55.8 MB/s 
  Using cached jedi-0.18.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=9d4b7cf6e339ffbc09188220d5d2a7f756919cfa0129f893402424238829f2cb
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
pd.set_option('display.max_columns', None)
from shapely.geometry import LineString
from keplergl import KeplerGl
## https://docs.kepler.gl/docs/keplergl-jupyter#6-match-config-with-data
from ast import literal_eval
import json

In [ ]:
input_file = 'route_plot'
html_output = 'tsp_output'

In [ ]:
df = pd.read_excel('%s.xlsx'%(input_file), sheet_name = 'Route_Info_Plot')
df.Route = df.Route.apply(literal_eval)
route_dict = df[["Rider_ID", "Route"]].set_index('Rider_ID').T.to_dict('records')[0]
df

,Rider_ID,Route
0,0,"[Kolkata, Uluberiya, Bārāsat, Agartala, Āīzawl..."


In [ ]:
lat_long = pd.read_excel('%s.xlsx'%(input_file), sheet_name = 'Cities_lat_long')
latlong_dict = lat_long.set_index('id').T.to_dict('list')
lat_long.head()

,id,lat,long
0,Kolkata,22.5411,88.3378
1,Delhi,28.6600,77.2300
2,Mumbai,18.9667,72.8333
3,Bangalore,12.9699,77.5980
4,Chennai,13.0825,80.2750


In [ ]:
line = []
for index, row in df.iterrows():
    lat_long_list = []
    for i in range(len(route_dict[row['Rider_ID']])):
        lat = latlong_dict[route_dict[row['Rider_ID']][i]][0]
        long = latlong_dict[route_dict[row['Rider_ID']][i]][1]
        lat_long_list.append((long,lat))
    line.append([row['Rider_ID'], LineString(lat_long_list)])

In [ ]:
line_df = pd.DataFrame([{'rider_id': line[i][0] ,'geometry':line[i][1].wkt} for i in range(len(line))])
line_df.to_csv('line_{0}.csv'.format(html_output))

In [ ]:
line_df.head()

,rider_id,geometry
0,0,"LINESTRING (88.3378 22.5411, 88.11 22.47, 88.4..."


In [ ]:
with open("config_optimal_{0}.json".format(html_output)) as f:
    config_data = json.load(f)
config_data

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '5d4o5hb',
     'type': 'geojson',
     'config': {'dataId': 'Line_data',
      'label': 'Line_data',
      'color': [18, 147, 154],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 

In [ ]:
map_1 = KeplerGl(config=config_data)
map_1.add_data(data=line_df, name='Line_data')
map_1.add_data(data=lat_long, name='lat_long')
map_1.add_data(data=df, name='all_features')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '5d4o5hb', 'type': …

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
map_1.save_to_html(config=map_1.config, file_name='{0}.html'.format(html_output))

Map saved to tsp_output.html!


In [ ]:
config = map_1.config
with open("config_optimal_{0}.json".format(html_output), "w") as out_file:
        json.dump(config, out_file)

In [ ]:
# from google.colab import output
# output.disable_custom_widget_manager()